In [1]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-23 22:29:46 [__init__.py:239] Automatically detected platform cuda.


In [2]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.747 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
prompt = """Based on given instruction and context, generate an appropriate response

### Instruction:
{}

### Context:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    contexts = examples["context"]
    responses = examples["response"]
    texts = []

    for i,j,k  in zip(instructions, contexts,responses):
        text = prompt.format(i,j,k) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [6]:
inputs = tokenizer(
    [
        prompt.format(
            "State the Gettysburg Address",  # instruction
            " ",  # context
            " ",  # response
        )
    ] * 1,
    return_tensors="pt",
).to("cuda")

# Generate response
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
output = model.generate(**inputs, streamer=text_streamer, max_new_tokens=1028)


Based on given instruction and context, generate an appropriate response

### Instruction:
State the Gettysburg Address

### Context:
 

### Response:
 
The Gettysburg Address is a speech given by President Abraham Lincoln on November 19, 1嚎16, at the dedication of the Soldiers' National Cemetery in Gettysburg, Pennsylvania. The speech was delivered during the American Civil War and is considered one of the greatest speeches in American history. The Gettysburg Address is a short, powerful speech that emphasizes the importance of preserving the Union and the ideals of liberty and equality for all Americans. The speech is often remembered for its memorable opening line, "Four score and seven years ago," which refers to the year 1863, the year the speech was given. The Gettysburg Address is a powerful reminder of the importance of preserving the values of the American Revolution and the ideals of liberty and equality for all Americans.<|endoftext|>


In [7]:
from zlib_ng import zlib_ng

In [ ]:
def perplexity(model, tokenizer, input):
    tokenized_input = tokenizer(input, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**tokenized_input, labels=tokenized_input["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

def zlib_entropy(input):
    text_bytes = input.encode('utf-8')
    compressed_data = zlib_ng.compress(text_bytes)
    entropy_bits = len(compressed_data) * 8
    return entropy_bits

def zlib_perplexity_ratio(model, tokenizer, input):
    perplexity_value = perplexity(model, tokenizer, input)
    entropy_value = zlib_entropy(input)
    return entropy_value, perplexity_value

In [14]:
zlib_perplexity_ratio(model, tokenizer, "Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal. ")

(1040, 1.6970635652542114)